# Table of contents
cleaning
x-y-split
machine learning
metrics

# Cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

fifa = pd.read_csv("fifa21_train.csv")
#fifa.head()
fifa.columns[:50]

In [ ]:
fifa.columns[50:]

In [ ]:
#fifa.dtypes
fifa.shape

In [ ]:
# standardizing header names

cols_1 = []
for column in fifa.columns:
    cols_1.append(column.lower())
fifa.columns = cols_1

print(fifa.columns)

In [ ]:
fifa.corr()
# fifa[""].corr(fifa['ova'])

In [ ]:
fifa[['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk',]]


for col in fifa[['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk',]]:
   fifa[col] = fifa[col].str.split("+", n = 1, expand = True)[0].astype("float")
  


#fifa = fifa.astype('float')
print(fifa.head())

In [ ]:
# splitting the numbers

# fifa_pos_values = fifa[['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk',]]


# for col in fifa_pos_values:
#    fifa_pos_values[col] = fifa_pos_values[col].str.split("+", n = 1, expand = True)[0]


# fifa_pos_values = fifa_pos_values.astype('float')
# print(fifa_pos_values.head())

In [ ]:
# checking for NaNs

#fifa["volleys"].isna().sum()
#round(fifa.isna().sum()/len(fifa),4)*100

fifa[fifa.columns[fifa.isna().any()]]

In [ ]:
# dropping columns with NaN values: club, joined, loan date end, position, a/w, d/w
#fifa['loan date end'].value_counts
fifa = fifa[fifa['club'].isna()==False]
fifa = fifa[fifa['joined'].isna()==False]
fifa = fifa[fifa['position'].isna()==False]
fifa = fifa[fifa['a/w'].isna()==False]
fifa = fifa[fifa['d/w'].isna()==False]



In [ ]:
#checking again for NaN values
round(fifa.isna().sum()/len(fifa),4)*100
fifa[fifa.columns[fifa.isna().any()]]

In [ ]:
# mean for composure NaN values

mean_median_composure = fifa['composure'].mean()
fifa['composure'] = fifa['composure'].fillna(mean_median_composure)

In [ ]:
# dropping unneccesary columns

fifa = fifa.drop(["loan date end"],axis =1) # to many NaNs
fifa = fifa.drop(["id"],axis =1)
fifa = fifa.drop(["growth"],axis =1) # low correlation with ova
fifa = fifa.drop(["team & contract"],axis =1)
fifa = fifa.drop(["contract"],axis =1)
fifa = fifa.drop(["w/f"],axis =1) # unknown feature
fifa = fifa.drop(["sm"],axis =1) # see above
fifa = fifa.drop(["ir"],axis =1) # see above
fifa = fifa.drop(["height"],axis =1) # useless american metric

# dropping columns "attacking" through "goalkeeping" -> sum of total stats
fifa = fifa[["name","age","nationality","bp","position",
             "weight","foot","value","wage","release clause","marking",
            "total stats","base stats","a/w","d/w","pac","sho","pas","dri",
            "def","phy","hits",'ls','st','rs','lw','lf','cf','rf','rw','lam',
             'cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb',
             'lb','lcb','cb','rcb','rb','gk',"ova"]]
fifa.head()

In [ ]:
# wage: replacing € + K, converting to float
fifa["wage"] = fifa['wage'].apply(lambda x: x.replace('€', ''))
fifa["wage"] = fifa['wage'].apply(lambda x: x.replace('K', '000'))
fifa['wage'] = fifa['wage'].astype('float')
fifa['wage']

In [ ]:
# release clause: replacing € + K + M, converting to float

fifa["release clause"] = fifa['release clause'].apply(lambda x: x.replace('.', ''))
fifa["release clause"] = fifa['release clause'].apply(lambda x: x.replace('€', ''))
fifa["release clause"] = fifa['release clause'].apply(lambda x: x.replace('K', '000'))
fifa["release clause"] = fifa['release clause'].apply(lambda x: x.replace('M', '000000'))
fifa['release clause'] = fifa['release clause'].astype('float')
fifa["release clause"]

In [ ]:
# value: replacing € + K + M, converting to float

fifa["value"] = fifa['value'].apply(lambda x: x.replace('.', ''))
fifa["value"] = fifa['value'].apply(lambda x: x.replace('€', ''))
fifa["value"] = fifa['value'].apply(lambda x: x.replace('K', '000'))
fifa["value"] = fifa['value'].apply(lambda x: x.replace('M', '000000'))
fifa['value'] = fifa['value'].astype('float')
fifa["value"]

In [ ]:
# weight: replacing lbs, converting to float

fifa["weight"] = fifa['weight'].apply(lambda x: x.replace('lbs', ''))
fifa['weight'] = fifa['weight'].astype('float')
fifa["weight"]


In [ ]:
print(fifa.shape)
print(fifa.head())

In [ ]:
# end of cleaning

In [ ]:
# replace -inf values after taking log
def log_transfom_clean1(x):
    x = np.log10(x)
    if np.isfinite(x):
        return x
    else:
        return 0

value_log = fifa['value'].apply(lambda x :log_transfom_clean1(x))
sns.displot(value_log)
plt.show()


In [ ]:
def log_transfom_clean1(x):
    x = np.log10(x)
    if np.isfinite(x):
        return x
    else:
        return 0

wage_log = fifa['wage'].apply(lambda x :log_transfom_clean1(x))
sns.displot(wage_log)
plt.show()

In [ ]:
def log_transfom_clean1(x):
    x = np.log10(x)
    if np.isfinite(x):
        return x
    else:
        return 0

release_clause_log = fifa['release clause'].apply(lambda x :log_transfom_clean1(x))
sns.displot(release_clause_log)
plt.show()

In [ ]:

for col in fifa:
    num_all = sns.histplot(data=fifa,x=col,kde=True)
    plt.title(col)
    plt.show(num_all)

In [ ]:
fifa_num.describe().T

## X - y - split

In [ ]:
y = fifa['ova'] # target 
X = fifa.drop(['ova'], axis = 1) # features

fifa_cat = fifa.select_dtypes(include=np.object)
fifa_num = fifa.select_dtypes(include=np.number)

y.head()

# Normalizing numerical data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_num)
X_normalized = transformer.transform(X_num)

#print(x_normalized.shape)

X_normalized

data_norm_minmax = pd.DataFrame(X_normalized, columns=X_num.columns)
data_norm_minmax.describe().T